# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 9 2023 10:36AM,254559,10,Clear-9009953,"ClearSpec, LLC",Released
1,Jan 9 2023 10:34AM,254558,20,ATR-I000043,"HVL, LLC dba Atrium Innovations",Released
2,Jan 9 2023 10:24AM,254555,10,8997621,Eywa Pharma Inc.,Released
3,Jan 9 2023 10:24AM,254555,10,8998196,Eywa Pharma Inc.,Released
4,Jan 9 2023 10:24AM,254555,10,9009419,Eywa Pharma Inc.,Released
5,Jan 9 2023 10:24AM,254552,15,9008875,"Brookfield Pharmaceuticals, LLC",Released
6,Jan 9 2023 10:24AM,254552,15,9009388,"Brookfield Pharmaceuticals, LLC",Released
7,Jan 9 2023 10:24AM,254552,15,9009389,"Brookfield Pharmaceuticals, LLC",Released
8,Jan 9 2023 10:24AM,254552,15,9009390,"Brookfield Pharmaceuticals, LLC",Released
9,Jan 9 2023 10:24AM,254552,15,9009391,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,254554,Released,23
33,254555,Released,3
34,254556,Released,5
35,254558,Released,1
36,254559,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254554,NaN,NaN,23.0
254555,NaN,NaN,3.0
254556,NaN,NaN,5.0
254558,NaN,NaN,1.0
254559,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254505,3.0,1.0,7.0
254511,0.0,0.0,8.0
254512,0.0,0.0,11.0
254516,0.0,2.0,7.0
254517,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254505,3,1,7
254511,0,0,8
254512,0,0,11
254516,0,2,7
254517,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254505,3,1,7
1,254511,0,0,8
2,254512,0,0,11
3,254516,0,2,7
4,254517,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254505,3,1,7
1,254511,,,8
2,254512,,,11
3,254516,,2,7
4,254517,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 9 2023 10:36AM,254559,10,"ClearSpec, LLC"
1,Jan 9 2023 10:34AM,254558,20,"HVL, LLC dba Atrium Innovations"
2,Jan 9 2023 10:24AM,254555,10,Eywa Pharma Inc.
5,Jan 9 2023 10:24AM,254552,15,"Brookfield Pharmaceuticals, LLC"
22,Jan 9 2023 10:21AM,254554,10,ISDIN Corporation
45,Jan 9 2023 10:17AM,254556,10,ISDIN Corporation
50,Jan 9 2023 10:17AM,254550,10,ISDIN Corporation
83,Jan 9 2023 10:16AM,254549,10,ISDIN Corporation
125,Jan 9 2023 10:16AM,254548,10,ISDIN Corporation
179,Jan 9 2023 10:05AM,254551,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 9 2023 10:36AM,254559,10,"ClearSpec, LLC",,,1
1,Jan 9 2023 10:34AM,254558,20,"HVL, LLC dba Atrium Innovations",,,1
2,Jan 9 2023 10:24AM,254555,10,Eywa Pharma Inc.,,,3
3,Jan 9 2023 10:24AM,254552,15,"Brookfield Pharmaceuticals, LLC",,,17
4,Jan 9 2023 10:21AM,254554,10,ISDIN Corporation,,,23
5,Jan 9 2023 10:17AM,254556,10,ISDIN Corporation,,,5
6,Jan 9 2023 10:17AM,254550,10,ISDIN Corporation,,,33
7,Jan 9 2023 10:16AM,254549,10,ISDIN Corporation,,,42
8,Jan 9 2023 10:16AM,254548,10,ISDIN Corporation,,,54
9,Jan 9 2023 10:05AM,254551,10,Bio-PRF,,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 10:36AM,254559,10,"ClearSpec, LLC",1,,
1,Jan 9 2023 10:34AM,254558,20,"HVL, LLC dba Atrium Innovations",1,,
2,Jan 9 2023 10:24AM,254555,10,Eywa Pharma Inc.,3,,
3,Jan 9 2023 10:24AM,254552,15,"Brookfield Pharmaceuticals, LLC",17,,
4,Jan 9 2023 10:21AM,254554,10,ISDIN Corporation,23,,
5,Jan 9 2023 10:17AM,254556,10,ISDIN Corporation,5,,
6,Jan 9 2023 10:17AM,254550,10,ISDIN Corporation,33,,
7,Jan 9 2023 10:16AM,254549,10,ISDIN Corporation,42,,
8,Jan 9 2023 10:16AM,254548,10,ISDIN Corporation,54,,
9,Jan 9 2023 10:05AM,254551,10,Bio-PRF,6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 10:36AM,254559,10,"ClearSpec, LLC",1,,
1,Jan 9 2023 10:34AM,254558,20,"HVL, LLC dba Atrium Innovations",1,,
2,Jan 9 2023 10:24AM,254555,10,Eywa Pharma Inc.,3,,
3,Jan 9 2023 10:24AM,254552,15,"Brookfield Pharmaceuticals, LLC",17,,
4,Jan 9 2023 10:21AM,254554,10,ISDIN Corporation,23,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 10:36AM,254559,10,"ClearSpec, LLC",1.0,NaN,NaN
1,Jan 9 2023 10:34AM,254558,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN
2,Jan 9 2023 10:24AM,254555,10,Eywa Pharma Inc.,3.0,NaN,NaN
3,Jan 9 2023 10:24AM,254552,15,"Brookfield Pharmaceuticals, LLC",17.0,NaN,NaN
4,Jan 9 2023 10:21AM,254554,10,ISDIN Corporation,23.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 9 2023 10:36AM,254559,10,"ClearSpec, LLC",1.0,0.0,0.0
1,Jan 9 2023 10:34AM,254558,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0
2,Jan 9 2023 10:24AM,254555,10,Eywa Pharma Inc.,3.0,0.0,0.0
3,Jan 9 2023 10:24AM,254552,15,"Brookfield Pharmaceuticals, LLC",17.0,0.0,0.0
4,Jan 9 2023 10:21AM,254554,10,ISDIN Corporation,23.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5599885,383.0,62.0,3.0
12,509059,3.0,0.0,0.0
15,254552,17.0,0.0,0.0
16,509065,2.0,0.0,0.0
19,763562,20.0,0.0,0.0
20,509075,2.0,0.0,0.0
22,254519,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5599885,383.0,62.0,3.0
1,12,509059,3.0,0.0,0.0
2,15,254552,17.0,0.0,0.0
3,16,509065,2.0,0.0,0.0
4,19,763562,20.0,0.0,0.0
5,20,509075,2.0,0.0,0.0
6,22,254519,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,383.0,62.0,3.0
1,12,3.0,0.0,0.0
2,15,17.0,0.0,0.0
3,16,2.0,0.0,0.0
4,19,20.0,0.0,0.0
5,20,2.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,383.0
1,12,Released,3.0
2,15,Released,17.0
3,16,Released,2.0
4,19,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,62.0,0.0,0.0,0.0,0.0,0.0,0.0
Released,383.0,3.0,17.0,2.0,20.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,62.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Released,383.0,3.0,17.0,2.0,20.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,62.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Released,383.0,3.0,17.0,2.0,20.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()